"""
This is our oroject mainfile
"""

Modules of python
-----------------

In [249]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from random import randint

Modules with own classes
------------------------

In [250]:
from glassdata import GlassData
from network import NeuralNetwork

---------------------------------------
Data-set on rho and ANN on molar volume
---------------------------------------

In [251]:
# Dataset of rho
filedbrho='DataBase/rho20oxides.csv'
dbrho=GlassData(filedbrho)
dbrho.info()
dbrho.bounds()

**************
Finished loading dataset
Nb of samples: 64421
Nb of components: 20
**************
Finished loading dataset
Nb of samples: 64421
Nb of components: 20


In [252]:
# Determination of the molar volume
dbrho.oxidemolarmass()
dbrho.molarmass()
dbrho.y=dbrho.MolarM/dbrho.y
dbrho.normalize_y()

In [253]:
# Loading of the ANN model
arch=[20,20,20]
nnmolvol=NeuralNetwork(dbrho.noxide,arch,'gelu','linear')
nnmolvol.compile(3.e-4)
nnmolvol.ArchName(arch)
nnmolvol.load('Models/nnmolarvol'+nnmolvol.namearch+'.h5')
nnmolvol.info()

**************


Model: "sequential_181"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_487 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_488 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_489 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_490 (Dense)               │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,283 (5.02 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

------------------------------------------------
Data-set on Young's modulus and ANN on Vt=E/(2G)
------------------------------------------------

In [254]:
filedbE='DataBase/E20oxides.csv'
dbE=GlassData(filedbE)
dbE.info()
dbE.bounds()

**************
Finished loading dataset
Nb of samples: 10846
Nb of components: 20
**************
Finished loading dataset
Nb of samples: 10846
Nb of components: 20


------------------------------
Loading of dissociation energy
------------------------------

In [255]:
datadisso=pd.read_csv('dissociationenergy.csv')
G=np.zeros(dbE.nsample)
for i in range(dbE.nsample):
    G[i]=np.sum(datadisso['G'].values*dbE.x[i,:])
#end for

In [256]:
# Determination of E/G and normalization
dbE.y=dbE.y/(2.*G)
dbE.normalize_y()

------------------------------
Loading of the ANN model on Vt
------------------------------

In [257]:
arch=[20,20,20]
nnmodelEsG=NeuralNetwork(dbE.noxide,arch,'gelu','linear')
nnmodelEsG.compile(1.e-4)
nnmodelEsG.ArchName(arch)
nnmodelEsG.load('Models/nnEsG'+nnmodelEsG.namearch+'.h5')
nnmodelEsG.info()

**************


Model: "sequential_262"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_986 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_987 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_988 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_989 (Dense)               │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,283 (5.02 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

---------------------------------------
Data-set on Tannealing=Tg and ANN model
---------------------------------------

In [258]:
# Data-set of Tannealing
filedbTannealing='DataBase/Tannealing20oxides.csv'
dbTannealing=GlassData(filedbTannealing)
dbTannealing.bounds()
dbTannealing.normalize_y()

**************
Finished loading dataset
Nb of samples: 6612
Nb of components: 20


In [259]:
# ANN model on Tannealing
# -----------------------
arch=[20,20,20]
nnTannealing=NeuralNetwork(dbTannealing.noxide,arch,'gelu','linear')
nnTannealing.compile(3.e-4)
nnTannealing.ArchName(arch)
nnTannealing.load('Models/nn'+dbTannealing.nameproperty+nnTannealing.namearch+'.h5')
nnTannealing.info()

**************


Model: "sequential_187"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_585 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_586 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_587 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_588 (Dense)               │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,283 (5.02 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

Data-set on Tmelt
-----------------

In [260]:
filedbTmelt='DataBase/Tmelt19oxides.csv'
dbTmelt=GlassData(filedbTmelt)
dbTmelt.info()
dbTmelt.bounds()
dbTmelt.normalize_y()

**************
Finished loading dataset
Nb of samples: 5996
Nb of components: 19
**************
Finished loading dataset
Nb of samples: 5996
Nb of components: 19


In [261]:
# ANN model on Tmelt
# ------------------
arch=[20,20,20]
nnTmelt=NeuralNetwork(dbTmelt.noxide,arch,'gelu','linear')
nnTmelt.compile(3.e-4)
nnTmelt.ArchName(arch)
nnTmelt.load('Models/nn'+dbTmelt.nameproperty+nnTmelt.namearch+'.h5')
nnTmelt.info()

**************


Model: "sequential_34"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_116 (Dense)               │ (None, 20)             │           400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_117 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_118 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_119 (Dense)               │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,263 (4.94 KB)

 Trainable params: 1,261 (4.93 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

------------------------------
Data-set on Tliq and ANN model
------------------------------

In [262]:
filedbTliq='DataBase\Tsoft20oxides.csv'
dbTliq=GlassData(filedbTliq)
dbTliq.info()
dbTliq.bounds()
dbTliq.normalize_y()

**************
Finished loading dataset
Nb of samples: 12531
Nb of components: 20
**************
Finished loading dataset
Nb of samples: 12531
Nb of components: 20


<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\vivgh\AppData\Local\Temp\ipykernel_58020\4115716478.py:1: SyntaxWarning: invalid escape sequence '\T'
  filedbTliq='DataBase\Tsoft20oxides.csv'


In [263]:
# ANN model on Tliq
# -----------------
arch=[32,32,32,32]
nnTliq=NeuralNetwork(dbTliq.noxide,arch,'gelu','linear')
nnTliq.compile(3.e-4)
nnTliq.ArchName(arch)
#modelfile='Models\nn'+dbTliq.nameproperty+nnTliq.namearch+'.h5'
modelfile='Models/nnTsoft3c20.h5'
nnTliq.load(modelfile)
nnTliq.info()

**************


Model: "sequential_47"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_172 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_173 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_174 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_175 (Dense)               │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,283 (5.02 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

------------------------------------------
Determination of the bounds for each oxide
------------------------------------------

# Algo genetique

## Variables utiles

In [282]:
rho_db = pd.read_csv(r'DataBase\rho20oxides.csv', header=0)
E_db = pd.read_csv('DataBase\\E20oxides.csv', header=0)
Tg_db = pd.read_csv('DataBase\\Tannealing20oxides.csv', header=0)
Tm_db = pd.read_csv('DataBase\\Tmelt19oxides.csv', header=0)

prop_min = np.array([min(rho_db['rho']),min(E_db['E']),min(Tg_db['Tannealing']),min(Tm_db['Tmelt'])])
prop_max = np.array([max(rho_db['rho']),max(E_db['E']),max(Tg_db['Tannealing']),max(Tm_db['Tmelt'])])

print(prop_min,prop_max)

[1789.99996185   37.49200058  549.73444214  961.68375244] [7072.0000267   110.4        1190.15       2053.15061035]


In [283]:
labels = dbrho.oxide
N_oxides = len(labels)
available_mat = ['SiO2', 'Al2O3', 'MgO', 'CaO', 'Na2O', 'K2O','ZnO', 'TiO2']
prop_label = ['rho','E','Tg','Tmelt']
columns = list(labels)+prop_label+['F']

#Contraintes
xmaxt=np.array([dbrho.xmax,dbE.xmax,dbTannealing.xmax,np.append(dbTmelt.xmax,1.),dbTliq.xmax])
xmax=np.zeros(dbrho.noxide)
for i in range(dbrho.noxide):
    if dbrho.oxide[i] in available_mat:
        xmax[i]=np.min(xmaxt[:,i])

xmin = np.zeros(dbrho.noxide)
xmin[list(dbrho.oxide).index('SiO2')] = 0.45
xmin[list(dbrho.oxide).index('Na2O')] = 0.1

#Parametres fitness function
#L'ordre des parametres est ['rho','E','Tg','Tmelt']
weight=[0,0.4,0,0.6]
minimize=[True,False,False,True]

N_generations = 200
N_population = 400

survivor_rate = 0.1
parent_rate = 0.5
child_rate = 0.7
mutation_rate = 0
immigration_rate = 1 - survivor_rate - child_rate

strategies = ['China','tournament']
N_tournament = 5
child_strategy = strategies[1]

#population = np.zeros((N_population,len(labels) + len(prop_label) + 1))

N_parents = int(parent_rate * N_population)
N_childs = int(child_rate * N_population)
N_mutants = int(0.1 * N_population)

epsilon = 0.05


## Creation de generations

In [266]:
def prop_calculation(composition):
    rho=dbrho.GlassDensity(nnmolvol,dbrho.oxide,composition)
    E=dbE.YoungModulus(nnmodelEsG,datadisso,dbE.oxide,composition)
    Tg=dbTannealing.physicaly(nnTannealing.model.predict(composition).transpose()[0,:])
    Tmelt=dbTmelt.physicaly(nnTmelt.model.predict(composition[:,:-1]).transpose()[0,:])
    return np.vstack((rho,E,Tg,Tmelt)).transpose()

In [267]:
def normalize(prop):
    #return (prop - prop.min(axis=0))/(prop.max(axis=0)-prop.min(axis=0))
    return (prop - prop_min)/(prop_max - prop_min)

In [268]:
#prop est une array avec les proprietes du verre normalisées, weight est le poids qu'on accorde
#à chacune des proprietes, et minize est une liste de booléens selon qu'on veuille minimiser
#ou maximiser une certaine variable
def fitness_func(prop_normalized,weight,minimize):
    rating = np.zeros(prop_normalized.shape[0])
    for i in range(len(weight)):
        if minimize[i]:
            rating += (1-prop_normalized[:,i])*weight[i]
        else:
            rating += prop_normalized[:,i]*weight[i]
    return rating

In [269]:
# Trie la population par F decroissant et renvoie cette population triée avec une nuovelle colonne qui represente 
# le fitness de chaque composition.
def stack_by_f(population,properties,F):
    population_info = np.column_stack((population,properties,F))
    sorted_arr = population_info[population_info[:, -1].argsort()][::-1]
    return sorted_arr

In [270]:
def init_properties(population):
    prop = prop_calculation(population)
    normalized_prop = normalize(prop)
    F = fitness_func(normalized_prop,weight,minimize)
    sorted_arr = stack_by_f(population, prop, F)
    return sorted_arr

In [271]:
def compute_properties(generation):
    population_sorted = init_properties(generation[:, :20])
    return population_sorted

In [272]:
def init_pop(N_population):
    population,_=dbrho.better_random_composition(N_population,xmin,xmax)
    population = init_properties(population)
    return population

In [273]:
def parent_choice (parents,strategie) :
    #Dans parents chaque individu est représenté par 21 floats dont le dernier est la valeur de fitness
    if strategie =='China' and N_parents>= 2*N_childs:
        dads = parents[::2]
        moms = parents[1::2]
        return (np.array(moms[:N_childs]),np.array(dads[:N_childs]))    
    if strategie =='tournament':
        N_tournament =5
        dads =[]
        moms =[]
        for i in range(N_childs):
            t = np.array([parents[i,:] for i in np.random.choice(N_parents,N_childs)])
            dad = t[np.argmax(t[:N_tournament,-1])]
            mom = t[np.argmax(t[N_tournament:,-1])]
            dads.append(dad)
            moms.append(mom)
        return (np.array(moms),np.array(dads))
    #Fallback
    t = np.array([parents[i] for i in np.random.choice(N_parents,N_childs*2)])
    return (np.array(t[N_childs:]),np.array(t[:N_childs]))

In [274]:
def population_selection(generation):
    survivors = generation[:int(N_population*survivor_rate)]
    dads,moms = parent_choice(generation,child_strategy)
    #to_be_mutated = sorted_population[int(N_population*survivor_rate):int(N_population*survivor_rate)+int(N_population*mutation_rate)]
    return survivors,dads,moms 

In [275]:
'''def crossover (parents) :
    #Dans parents chaque individu est représenté par 20 premiers floats et le dernier est la valeur de fitness
    childs = np.array([[0.] * len(parents[0])] * N_childs)
    for i in range (N_childs) :
        i1 = randint(0, N_parents-1)
        i2 = randint(0, N_parents-2)
        if i2 == i1 : #problème si deux fois le même parent !!
            i2 += 1
        w1 = parents[i1, -1] / (parents[i1, -1] + parents[i2, -1]) #poids du parent 1
        w2 = parents[i2, -1] / (parents[i1, -1] + parents[i2, -1]) #poids du parent 2
        childs[i] = (w1 * parents[i1] + w2 * parents[i2]) #moyenne pondérée
    return (childs)'''

'def crossover (parents) :\n    #Dans parents chaque individu est représenté par 20 premiers floats et le dernier est la valeur de fitness\n    childs = np.array([[0.] * len(parents[0])] * N_childs)\n    for i in range (N_childs) :\n        i1 = randint(0, N_parents-1)\n        i2 = randint(0, N_parents-2)\n        if i2 == i1 : #problème si deux fois le même parent !!\n            i2 += 1\n        w1 = parents[i1, -1] / (parents[i1, -1] + parents[i2, -1]) #poids du parent 1\n        w2 = parents[i2, -1] / (parents[i1, -1] + parents[i2, -1]) #poids du parent 2\n        childs[i] = (w1 * parents[i1] + w2 * parents[i2]) #moyenne pondérée\n    return (childs)'

In [276]:
# températures de fusion des oxydes (en °C) : 
# [1610, 2045, 2852, 2580]
# dureté des oxydes :
# [7, 9, 5.5]
# TiO2 est un agent nucléant utilisé pour les plaques vitrocéramiques (très faible coeff de dilatation thermique).
# source : L'élémentarium

# fondants = ['Na2O', 'K2O', 'MgO']
# stabilisants = ['CaO', 'ZnO']
# ZnO augmente l'élasticité
# source : https://lasirene.e-monsite.com/pages/le-verre/-.html

# formateurs = ['SiO2']
# SiO2 augmente la dureté du verre
# fondants = ['Na2O', 'K2O', 'MgO']
# stabilisants = ['CaO', 'ZnO']
# source : https://infovitrail.com/contenu.php/fr/d/---la-composition-du-verre/e9b609c9-91f5-4a08-86a6-6112dc12b66d

# fondants = ['Na2O', 'K2O'] + ['CaO', 'MgO'] (mais moins bien que les deux premiers)
# modificateurs = ['CaO', 'MgO', 'Al2O3'] #augmentent les propriétés de durabilité chimique et mécanique (E ??)
# formateurs = ['SiO2'] #essentiels
# source : Franck

fondants = [False, False, False, True, True, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False] # diminuent Tm
durability = [True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False] #augmentent E
other = [False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False]

def crossover (mom, dad) :
    print(mom)
    childs = np.zeros((len(mom), len(mom[0])))
    for i in range (N_childs) :
        if mom[i, 21] < dad[i, 21] : #choix du meilleur E
            bestE = dad[i]
        else :
            bestE = mom[i]
        if mom[i, 23] < dad[i, 23] : #choix du meilleur Tm
            bestTm = mom[i]
        else :
            bestTm = dad[i]
        childs[i][fondants] = bestTm[fondants]
        childs[i][durability] = bestE[durability]
        childs[i][other] = (mom[i][other] + dad[i][other])/2
        sum = np.sum(childs[i])
        childs[i] = childs[i] / sum
    return (childs)

In [277]:
def mutation (mutants) :
    #les mutants sont les meilleures compositions entre 40% et 50%
    for j in range (N_mutants) :
        iplus = randint(0, 19) #choix de l'oxyde qui gagne epsilon
        imoins = randint(0, 19) #choix de l'oxyde qui perd epsilon
        if imoins == iplus : #problème si deux fois le même oxyde !!
            imoins = (1 + imoins)%19
        mutantPlus = mutants[j, iplus]
        mutantMoins = mutants[j, imoins]
        if (mutantMoins > epsilon + xmin[j]) and (mutantPlus < xmax[j] - epsilon) :
            mutants[j, iplus] = mutantPlus + epsilon
            mutants[j, imoins] = mutantMoins - epsilon
    return (mutants)

In [278]:
def new_generation(old_generation):
    survivors,dads,moms = population_selection(old_generation)
    child = crossover(dads,moms)
    immigrants = init_pop(N_population - (len(survivors) + len(child)))
    new_population = np.vstack((np.vstack((survivors,child)),immigrants))
    new_population = compute_properties(new_population)
    return new_population

In [279]:
def evolution(generation,N):
    for _ in range(N):
        generation = new_generation(generation)
    return generation

In [280]:
initial_pop = init_pop(N_population)
compositions = evolution(initial_pop,N_generations)

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 958us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 917us/step
[[5.82832114e-01 0.00000000e+00 1.43368963e-02 ... 7.87526829e+02
  1.48007423e+03 5.02395476e-01]
 [4.50003616e-01 0.00000000e+00 4.32399823e-03 ... 7.76644606e+02
  1.36042506e+03 5.78347736e-01]
 [4.51996650e-01 0.00000000e+00 3.53223251e-02 ... 8.05379802e+02
  1.44609777e+03 5.41622222e-01]
 ...
 [4.50386201e-01 0.00000000e+00 5.01245240e-02 ... 8.47602250e+02
  1.41536176e+03 5.90502820e-01]
 [4.50694621e-01 0.00000000e+00 6.49914580e-03 ... 7.68908531e+02
  1.45624942e+03 5.54005305e-01]
 [6.28382609e-01 0.00000000e+00 4.93747210e-05 ... 7.09369846e+02
  1.49677064e+03 4.64487288e-01]]
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
25/25 

In [281]:
df = pd.DataFrame(compositions,columns=columns)
#df.to_csv('generation_final.csv')
df

,SiO2,B2O3,Al2O3,MgO,CaO,BaO,Li2O,Na2O,K2O,ZnO,...,TiO2,GeO2,ZrO2,P2O5,V2O5,rho,E,Tg,Tmelt,F
0,0.387864,0.0,1.531366e-33,2.267628e-11,5.415683e-12,0.0,0.0,0.235009,0.228742,0.119931,...,0.028454,0.0,0.0,0.0,0.0,2789.203638,54.370357,567.095493,872.713125,0.911512
1,0.371391,0.0,3.740839e-06,2.258505e-11,5.393895e-12,0.0,0.0,0.234064,0.227821,0.121400,...,0.045321,0.0,0.0,0.0,0.0,2805.798520,48.497586,565.384650,852.780240,0.910012
2,0.375514,0.0,1.482606e-33,2.272454e-11,5.427209e-12,0.0,0.0,0.235509,0.229228,0.116727,...,0.043021,0.0,0.0,0.0,0.0,2797.484200,48.359532,564.651526,852.382212,0.909925
3,0.376884,0.0,3.796170e-06,2.273769e-11,5.430348e-12,0.0,0.0,0.235646,0.229361,0.115965,...,0.042141,0.0,0.0,0.0,0.0,2795.640540,48.539942,564.704923,853.350590,0.909710
4,0.373028,0.0,3.757329e-06,2.250369e-11,5.374463e-12,0.0,0.0,0.233221,0.227001,0.123031,...,0.043716,0.0,0.0,0.0,0.0,2806.444572,49.470694,565.978182,857.348132,0.909333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0.693453,0.0,8.869543e-02,8.852685e-02,1.382870e-02,0.0,0.0,0.114020,0.000008,0.000088,...,0.001380,0.0,0.0,0.0,0.0,2456.620644,75.313507,943.767701,1956.383957,0.174677
796,0.763916,0.0,3.526388e-02,1.340015e-02,3.051337e-02,0.0,0.0,0.101568,0.015944,0.005265,...,0.034130,0.0,0.0,0.0,0.0,2455.233610,73.029067,862.251788,1980.284485,0.150893
797,0.798757,0.0,1.343547e-03,7.463200e-04,3.972923e-10,0.0,0.0,0.100019,0.000315,0.098553,...,0.000267,0.0,0.0,0.0,0.0,2552.534832,69.463024,841.746714,1992.955730,0.131823
798,0.813093,0.0,1.128040e-04,4.527625e-02,3.270523e-02,0.0,0.0,0.100223,0.000005,0.006647,...,0.001938,0.0,0.0,0.0,0.0,2397.842924,70.578563,837.208285,2034.264661,0.104605
